In [11]:
import os
import cv2
import time
import pickle
import numpy as np
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

import numpy as np
import matplotlib.pyplot as plt

### 從資料夾讀取圖片，轉換 train 跟 test 資料，轉成 NumPy 數組，並且標準化資料

In [12]:
path = r'C:\Users\amysa\anaconda3\envs\cv\00\Machine Learning\Potato_Disease_Detection\potato_disease_Dataset\train'
data,label = [],[]
for root, dirs, files in os.walk(path):
    key = os.path.basename(root)
    for file in files:
        full_file_path = os.path.join(root,file)
        img = cv2.imread(full_file_path)
        img = cv2.resize(img,(256,256))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        data.append(img)
        label.append(key)

x_train = np.array(data)
y_train = np.array(label)

In [13]:
path = r'C:\Users\amysa\anaconda3\envs\cv\00\Machine Learning\Potato_Disease_Detection\potato_disease_Dataset\test'
data,label = [],[]
for root, dirs, files in os.walk(path):
    key = os.path.basename(root)
    for file in files:
        full_file_path = os.path.join(root,file)
        img = cv2.imread(full_file_path)
        img = cv2.resize(img,(256,256))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        data.append(img)
        label.append(key)

x_test = np.array(data)
y_test = np.array(label)
print(x_test)

[[[[157 156 172]
   [163 162 178]
   [165 164 180]
   ...
   [196 196 208]
   [190 190 202]
   [183 183 195]]

  [[164 163 179]
   [168 167 183]
   [168 167 183]
   ...
   [191 191 203]
   [189 189 201]
   [187 187 199]]

  [[166 165 181]
   [168 167 183]
   [167 166 182]
   ...
   [188 188 200]
   [191 191 203]
   [194 194 206]]

  ...

  [[106 104 118]
   [111 109 123]
   [123 121 135]
   ...
   [144 142 155]
   [156 154 167]
   [168 166 179]]

  [[104 102 116]
   [107 105 119]
   [123 121 135]
   ...
   [135 133 146]
   [142 140 153]
   [149 147 160]]

  [[102 100 114]
   [101  99 113]
   [120 118 132]
   ...
   [148 146 159]
   [141 139 152]
   [134 132 145]]]


 [[[187 186 200]
   [174 173 187]
   [168 167 181]
   ...
   [145 143 156]
   [127 125 138]
   [144 142 155]]

  [[180 179 193]
   [201 200 214]
   [224 223 237]
   ...
   [153 151 164]
   [140 138 151]
   [130 128 141]]

  [[198 197 211]
   [172 171 185]
   [176 175 189]
   ...
   [145 143 156]
   [148 146 159]
   [132 130

In [14]:
# Normalize Data
x_train = x_train / 255.0
x_test = x_test / 255.0

#### 將圖的數據展平並 encode labels，最後進行PCA降維(KNN在高維空間下，距離計算會變得較不可靠)

In [26]:
x_train = np.array(x_train).reshape(len(x_train),-1)
x_test = np.array(x_test).reshape(len(x_test),-1)

# 檢查展平後的形狀
print('Shape of x_train:', x_train.shape)
print('Shape of x_test:', x_test.shape)

#Encode labels from string to int
le = preprocessing.LabelEncoder()
labelEnc_train = le.fit_transform(y_train)
labelEnc_test = le.fit_transform(y_test)
le.fit(list(y_train) + list(y_test))  #Let LabelEncoder knows all the labels
labelEnc_train.shape

# PCA降維
pca = PCA(n_components=100)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)
print('complete')

Shape of x_train: (4200, 196608)
Shape of x_test: (1200, 196608)
complete


### KNN model

In [25]:
# 創建 KNN 模型
model = KNeighborsClassifier(n_neighbors=5)

# 使用展平後的訓練數據進行訓練，並使用 tqdm 來顯示訓練進度
with tqdm(total=len(x_train_pca)) as pbar:
    model.fit(x_train_pca, labelEnc_train)
    pbar.update(len(x_train_pca))


# 使用展平後的測試數據進行預測，並使用 tqdm 來顯示預測進度
with tqdm(total=len(x_test_pca)) as pbar:
    x_pred = model.predict(x_test_pca)
    pbar.update(len(x_test_pca))


# 計算並輸出準確率
accuracy = accuracy_score(labelEnc_test, x_pred)
print("Accuracy: ", accuracy * 100)

100%|████████████████████████████████████████████████████████████████████████████| 1200/1200 [00:00<00:00, 3399.88it/s]

Accuracy:  87.25
